In [1]:
import pandas as pd
import numpy as np

In [2]:
num_of_criteria = 6
importance_scale_width = 5
number_of_evaluators = 2

cols = num_of_criteria * (num_of_criteria-1) / 2
rows = importance_scale_width + (importance_scale_width - 1)

In [5]:
path = '../g-aph-matrix-results.xlsx'
results_df = pd.read_excel(path, header=0)
results_df = results_df.fillna(0)
results_df

,C-R,EZ-R,VJZ-R,JZ-R,SZ-R,PZ,SZ-L,JZ-L,VJZ-L,EZ-L,C-L
0,C1,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,C2
1,C1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,C3
2,C1,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,C4
3,C1,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,C5
4,C1,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,C6
5,C2,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,C3
6,C2,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,C4
7,C2,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,C5
8,C2,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,C6
9,C3,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,C4


In [6]:
ling_scale = ['Podjednaka znacajnost', 'Srednja znacajnost', 'Jaka znacajnost', 'Veoma jaka znacajnost', 'Ekstremna znacajnost']
scale = {'Linguistic Scale': ling_scale, 'Code': ['PZ', 'SZ', 'JZ', 'VJZ', 'EZ'], 'Grey Numbers': [[1, 2], [2, 4], [4, 6], [6, 8], [8, 10]]}
scale_df = pd.DataFrame(data=scale)
scale_df

,Linguistic Scale,Code,Grey Numbers
0,Podjednaka znacajnost,PZ,"[1, 2]"
1,Srednja znacajnost,SZ,"[2, 4]"
2,Jaka znacajnost,JZ,"[4, 6]"
3,Veoma jaka znacajnost,VJZ,"[6, 8]"
4,Ekstremna znacajnost,EZ,"[8, 10]"


In [7]:
def fill_diagonal(matrix, fill_value):
    matrix_dim = matrix.shape[0]
    for i in range(matrix_dim):
        for j in range(matrix_dim):
            if i == j:
                matrix[i,j] = fill_value

def print_grey_num(grey_num):
    return '[ ' + str(grey_num[0]) + ', ' + str(grey_num[1]) + ' ]'
                
def print_matrix(matrix):
    matrix_dim = matrix.shape[0]
    for i in range(matrix_dim):
        for j in range(matrix_dim):
            print(print_grey_num(matrix[i,j,:]), end =" ")
        print("\n")

In [8]:
aph_matrix = np.zeros((num_of_criteria, num_of_criteria, 2), dtype=float)
fill_diagonal(aph_matrix, [1.0, 1.0])
print_matrix(aph_matrix)

[ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] [ 0.0, 0.0 ] 

[ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 0.0, 0.0 ] [ 1.0, 1.0 ] 



In [9]:
def get_aph_matrix_indices_for_criteria(right_criteria_str, left_criteria_str):
    # remove 'C' from the begining
    right_criteria = right_criteria_str[1:len(right_criteria_str):1]
    left_criteria = left_criteria_str[1:len(left_criteria_str):1]
    return int(right_criteria)-1, int(left_criteria)-1

def get_reciprocal_grey_num(grey_num):
    return np.array([1/grey_num[1], 1/grey_num[0]])

def update_aph_matrix(grey_num_for_right_c, row_ind, col_ind):
    grey_num_for_left_c = get_reciprocal_grey_num(grey_num_for_right_c)
    aph_matrix[row_ind, col_ind] = np.around(grey_num_for_right_c, 3)
    aph_matrix[col_ind, row_ind] = np.around(grey_num_for_left_c, 3)

In [10]:
def get_code_and_side(axes_label):
    split_label = axes_label.split('-')
    if len(split_label) > 1:
        return split_label[0], split_label[1] #JZ, R
    return split_label[0], None # PZ

def get_grey_number_for_code(ling_scale_code):
    grey_num = list(scale_df[scale_df['Code'] == ling_scale_code]['Grey Numbers'])[0]
    return np.array(grey_num)

def left_c_is_more_important(side):
    return side != None and side == 'L'

def calc_grey_num_for_right_c_based_on_comparison(criteria_comparison):
    criteria_comparison = criteria_comparison[criteria_comparison!=0]
    length_of_series = criteria_comparison.shape[0]
    axes = criteria_comparison.axes[0]
    
    grey_num_for_right_criteria = np.array([0.0, 0.0])
    
    for i in range(1, length_of_series-1): #skip first and last 
        axes_label = axes[i] #JZ-R
        ling_scale_code, more_important_c_side = get_code_and_side(axes_label)
        grey_number_for_code = get_grey_number_for_code(ling_scale_code)
        
        if left_c_is_more_important(more_important_c_side):
            grey_number_for_code = get_reciprocal_grey_num(grey_number_for_code)
       
        num_of_evaluation_with_code = int(criteria_comparison[i])
        grey_num_for_right_criteria +=  grey_number_for_code * num_of_evaluation_with_code
    
    return grey_num_for_right_criteria / number_of_evaluators

In [11]:
num_of_comparisons = results_df.shape[0]

for i in range(num_of_comparisons):
    criteria_comparison = results_df.loc[i,:]
    right_criteria = criteria_comparison['C-R']
    left_criteria = criteria_comparison['C-L']
    grey_num_for_right_c = calc_grey_num_for_right_c_based_on_comparison(criteria_comparison)
    row_ind, col_ind = get_aph_matrix_indices_for_criteria(right_criteria, left_criteria)
    update_aph_matrix(grey_num_for_right_c, row_ind, col_ind)


In [12]:
print_matrix(aph_matrix)

[ 1.0, 1.0 ] [ 4.0, 6.0 ] [ 1.5, 3.0 ] [ 1.0, 2.0 ] [ 1.125, 2.25 ] [ 1.5, 3.0 ] 

[ 0.167, 0.25 ] [ 1.0, 1.0 ] [ 0.625, 1.25 ] [ 2.0, 4.0 ] [ 5.0, 7.0 ] [ 3.0, 5.0 ] 

[ 0.333, 0.667 ] [ 0.8, 1.6 ] [ 1.0, 1.0 ] [ 2.0, 4.0 ] [ 0.167, 0.25 ] [ 0.146, 0.208 ] 

[ 0.5, 1.0 ] [ 0.25, 0.5 ] [ 0.25, 0.5 ] [ 1.0, 1.0 ] [ 2.125, 3.25 ] [ 0.25, 0.5 ] 

[ 0.444, 0.889 ] [ 0.143, 0.2 ] [ 4.0, 6.0 ] [ 0.308, 0.471 ] [ 1.0, 1.0 ] [ 0.167, 0.25 ] 

[ 0.333, 0.667 ] [ 0.2, 0.333 ] [ 4.8, 6.857 ] [ 2.0, 4.0 ] [ 4.0, 6.0 ] [ 1.0, 1.0 ] 

